In [17]:
%run /home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/modulos/funciones/02_funciones_autonomia_economica.ipynb
#%run /Users/danielarisaro/Documents/Fundar/indice-mercado-trabajo-ingresos/modulos/funciones/02_funciones_autonomia_economica.ipynb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import pickle
warnings.filterwarnings('ignore')

In [18]:
# Definimos path
pathdata = '/home/daniu/Documentos/fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/Users/danielarisaro/Documents/Fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/'

In [19]:
with open(pathdata + 'modulos/diccionarios/' + 'diccionario_aglomerados.pickle', 'rb') as file:
    dict_cod_aglomerado = pickle.load(file)

with open(pathdata + 'modulos/diccionarios/' + 'diccionario_provincia.pickle', 'rb') as file:
    dict_cod_provincia = pickle.load(file)

with open(pathdata + 'modulos/diccionarios/' + 'diccionario_aglomerado_provincia.pickle', 'rb') as file:
    map_aglomerado_provincia = pickle.load(file)

with open(pathdata + 'modulos/diccionarios/' + 'diccionario_aglomerado_region.pickle', 'rb') as file:
    map_aglomerado_region = pickle.load(file)

with open(pathdata + 'modulos/diccionarios/' + 'diccionario_provincia_region.pickle', 'rb') as file:
    map_provincia_region = pickle.load(file)

In [20]:
df_people_pool = pd.read_csv(pathdata + 'data_output/Base_pool_individuos_solo_con_replicas_actuales.csv', low_memory=False, index_col=0)
df_houses_pool = pd.read_csv(pathdata + 'data_output/Base_pool_hogares_solo_con_replicas_actuales.csv', low_memory=False, index_col=0)

In [21]:
tasa, size, ratio, error = poblacion_inactiva_sin_ingresos(df_people_pool, tipo='Urbano', base='Individual', confidence_level=0.95, pool=True)
dependientes_sin_ingreso = pd.concat([tasa, size, ratio, error], axis=1)

In [22]:
dependientes_sin_ingreso.to_pickle(pathdata + 'data_output/df_dependientes_sin_ingresos_pool.pickle')

In [23]:
tabla_pob_inactiva, tabla_pob_inactiva_porcentaje, ratio_pob_inactiva = tabla_ingresos_inactivos_no_estudiantes(df_people_pool, pool=True)

In [24]:
tabla_pob_inactiva

In [25]:
tabla_pob_inactiva_porcentaje

In [26]:
ratio_pob_inactiva

,Ingreso > 0,Ingreso = 0,Ingreso = -9
Rango edad,,,
14-29,43.083401,213.478580,457.065866
30-45,221.641890,309.725462,216.624886
46-65,156.106555,110.884525,113.888920
65+,84.359985,75.180247,88.501539


In [27]:
df_temp = df_people_pool[df_people_pool['CH06'] >= 14].copy()
df_inactivos = df_temp[(df_temp['ESTADO']==3) & df_temp['CAT_INAC']!=3][['PONDII_new', 'PONDERA_new', 'CH04', 'P47T', 'CH06']]


In [28]:
import plotly.graph_objects as go


In [29]:
import plotly.express as px


In [30]:
fig = px.histogram(df_inactivos, x="P47T", y="PONDII_new", color="CH04", marginal="rug",
                   hover_data=df_inactivos.columns)

file_path = pathdata + 'figs/'
filename = 'panel-inactivos-pool.html'
fig.write_html(file_path + filename)

In [31]:
fig = px.histogram(df_inactivos[df_inactivos['P47T']==0], x="CH06", y="PONDERA_new", color="CH04", marginal="rug",
                   hover_data=df_inactivos[df_inactivos['P47T']==0].columns)

file_path = pathdata + 'figs/'
filename = 'panel-inactivos-pool_sin_ingreso.html'
fig.write_html(file_path + filename)

In [32]:
fig = px.histogram(df_inactivos[df_inactivos['P47T']>0], x="CH06", y="PONDERA_new", color="CH04", marginal="rug",
                   hover_data=df_inactivos[df_inactivos['P47T']>0].columns)

file_path = pathdata + 'figs/'
filename = 'panel-inactivos-pool_con_ingreso.html'
fig.write_html(file_path + filename)